In [1]:
import sys
# Uncomment if tornet isn't installed in your environment or in your path already
#sys.path.append('../')  

import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tornet.data.tf.loader import create_tf_dataset 
from tornet.data.constants import ALL_VARIABLES

In [5]:
data_root = 'C:/Users/mjhig/tornet_2013'
from tornet.data.keras.loader import KerasDataLoader


data_type='test'
years = [2013,2014,2015,2016,2017,2018]

catalog_path = os.path.join(data_root,'catalog.csv')
if not os.path.exists(catalog_path):
    raise RuntimeError('Unable to find catalog.csv at '+data_root)
        
catalog = pd.read_csv(catalog_path,parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type']==data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1,random_state=123)
file_list = [os.path.join(data_root,f) for f in catalog.filename]

ds_test = KerasDataLoader(data_root=data_root,
                         data_type='test',
                         catalog=catalog,
                         workers = 4,
                         use_multiprocessing = True)

In [41]:
# import tornet.data.preprocess as pp
# from tornet.data import preprocess as tfpp

# ds_test = ds_test.map(lambda d: pp.add_coordinates(d,include_az=False,backend=tf))
# ds_test = ds_test.map(pp.remove_time_dim)
# ds_test = ds_test.map(tfpp.split_x_y)
# ds_test = ds_test.prefetch(tf.data.AUTOTUNE)    
# ds_test = ds_test.batch(32)

In [1]:
from tensorflow.keras.models import load_model
def replace_nan(x):
    return tf.where(tf.math.is_nan(x), -3.0, x)


model = load_model('tornado_detector_baseline.keras')

In [ ]:
import tornet.models as k
dir(k)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'keras']

In [2]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ DBZ (InputLayer)    │ (None, None,      │          0 │ -                 │
│                     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VEL (InputLayer)    │ (None, None,      │          0 │ -                 │
│                     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ KDP (InputLayer)    │ (None, None,      │          0 │ -                 │
│                     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ RHOHV (InputLayer)  │ (None, None,      │          0 │ -                 │
│                     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ZDR (InputLayer)    │ (None, None,      │          0 │ -                 │
│                     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ WIDTH (InputLayer)  │ (None, None,      │          0 │ -                 │
│                     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_DBZ       │ (None, None,      │          0 │ DBZ[0][0]         │
│ (Normalization)     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_VEL       │ (None, None,      │          0 │ VEL[0][0]         │
│ (Normalization)     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_KDP       │ (None, None,      │          0 │ KDP[0][0]         │
│ (Normalization)     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_RHOHV     │ (None, None,      │          0 │ RHOHV[0][0]       │
│ (Normalization)     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_ZDR       │ (None, None,      │          0 │ ZDR[0][0]         │
│ (Normalization)     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Normalize_WIDTH     │ (None, None,      │          0 │ WIDTH[0][0]       │
│ (Normalization)     │ None, 2)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Concatenate1        │ (None, None,      │          0 │ Normalize_DBZ[0]… │
│ (Concatenate)       │ None, 12)         │            │ Normalize_VEL[0]… │
│                     │                   │            │ Normalize_KDP[0]… │
│                     │                   │            │ Normalize_RHOHV[… │
│                     │                   │            │ Normalize_ZDR[0]… │
│                     │                   │            │ Normalize_WIDTH[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ isnan_4 (Isnan)     │ (None, None,      │          0 │ Concatenate1[0][… │
│                     │ None, 12)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ where_4 (Where)     │ (None, None,      │          0 │ isnan_4[0][0],    │
│                     │ None, 12)         │            │ Concatenate1[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 4,665,409 (17.80 MB)

 Trainable params: 4,665,409 (17.80 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
import numpy as np
from tensorflow.keras.metrics import AUC, Precision, Recall, BinaryAccuracy

metrics = [
    AUC(name="roc_auc"),                # ROC AUC Score
    AUC(curve="PR", name="pr_auc"),     # Precision-Recall AUC (Better for imbalanced data)
    Precision(name="precision"),        # Precision
    Recall(name="recall"),              # Recall
    BinaryAccuracy(name="accuracy")     # Standard Accuracy
]

# Compile the model with the correct metrics
model.compile(metrics=metrics)
results = model.evaluate(ds_test, steps=10, return_dict=True)
print(results)  # Outputs evaluation metrics as a dictionary


NameError: name 'ds_test' is not defined

In [6]:
from tornet.metrics.keras.metrics import AUC,BinaryAccuracy,Precision
metrics = [AUC(curve='PR',name='PRAUC'),
            BinaryAccuracy(name='accuracy'),
           
           ]
model.compile(metrics=metrics)

# steps=10 for demo purposes
model.evaluate(ds_test,steps=10)

NameError: name 'ds_test' is not defined

In [55]:
model.predict(ds_test,steps=10)

c:\Users\mjhig\anaconda3\envs\tornet-torch\lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: {'DBZ': 'DBZ', 'VEL': 'VEL', 'KDP': 'KDP', 'RHOHV': 'RHOHV', 'ZDR': 'ZDR', 'WIDTH': 'WIDTH', 'range_folded_mask': 'range_folded_mask', 'coordinates': 'coordinates'}. Received: the structure of inputs={'DBZ': '*', 'VEL': '*', 'KDP': '*', 'RHOHV': '*', 'ZDR': '*', 'WIDTH': '*', 'range_folded_mask': '*', 'label': '*', 'category': '*', 'event_id': '*', 'ef_number': '*', 'az_lower': '*', 'az_upper': '*', 'rng_lower': '*', 'rng_upper': '*', 'time': '*', 'tornado_start_time': '*', 'tornado_end_time': '*', 'coordinates': '*'}
  warnings.warn(


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data_11:0", shape=(32,), dtype=float32). Expected shape (None, None, None, 2), but input has incompatible shape (32,)[0m

Arguments received by Functional.call():
  • inputs={'DBZ': 'tf.Tensor(shape=(32, 120, 240, 2), dtype=float32)', 'VEL': 'tf.Tensor(shape=(32, 120, 240, 2), dtype=float32)', 'KDP': 'tf.Tensor(shape=(32, 120, 240, 2), dtype=float32)', 'RHOHV': 'tf.Tensor(shape=(32, 120, 240, 2), dtype=float32)', 'ZDR': 'tf.Tensor(shape=(32, 120, 240, 2), dtype=float32)', 'WIDTH': 'tf.Tensor(shape=(32, 120, 240, 2), dtype=float32)', 'range_folded_mask': 'tf.Tensor(shape=(32, 120, 240, 2), dtype=float32)', 'label': 'tf.Tensor(shape=(32,), dtype=uint8)', 'category': 'tf.Tensor(shape=(32,), dtype=int64)', 'event_id': 'tf.Tensor(shape=(32,), dtype=int64)', 'ef_number': 'tf.Tensor(shape=(32,), dtype=int64)', 'az_lower': 'tf.Tensor(shape=(32,), dtype=float32)', 'az_upper': 'tf.Tensor

In [12]:
model.history